In [68]:
from dotenv import load_dotenv
load_dotenv()


True

#can 

In [81]:

# Create server parameters for stdio connection
from mcp import ClientSession, StdioServerParameters 
from mcp.client.sse import sse_client
from langchain_mcp_adapters.prompts import load_mcp_prompt
from langchain_mcp_adapters.resources import load_mcp_resources
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent

from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o")

server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your math_server.py file
    args=[r"C:\Users\Afzal\Desktop\mcp\langraph_mcp\poject00\LangGraph_MCP\mcp_server.py"],
)

async with sse_client(url="http://127.0.0.1:8001/sse") as streams:
    async with ClientSession(*streams) as session:
        # Initialize the connection
        await session.initialize()

        # Get tools
        tools = await load_mcp_tools(session)
                # 2) wrap your resources (including dynamic ones) as tools
        resource_uris = ["resource://some_static_resource", "greeting://{name}"]
        resource_tools = await load_mcp_resources(session, resource_uris)
        # resources = await load_mcp_resources(session)
        # #prompt = await load_mcp_prompt(session)
        all_tools = tools + resource_tools

        # Create and run the agent
        agent = create_react_agent(model= model, tools=tools)
        agent_response = await agent.ainvoke({"messages": """First compute (3 + 5) × 12.  """})
        

In [82]:
for m in agent_response["messages"]:
    m.pretty_print()

================================ Human Message =================================

First compute (3 + 5) × 12.  
================================== Ai Message ==================================
Tool Calls:
  add (call_fX6q9mw8g6DK40ZdPPUkI4lu)
 Call ID: call_fX6q9mw8g6DK40ZdPPUkI4lu
  Args:
    a: 3
    b: 5
  multiply (call_7K3t1f4KpVwkRgN0WMl3kybm)
 Call ID: call_7K3t1f4KpVwkRgN0WMl3kybm
  Args:
    a: 8
    b: 12
================================= Tool Message =================================
Name: add

8
================================= Tool Message =================================
Name: multiply

96
================================== Ai Message ==================================

The result of \( (3 + 5) \times 12 \) is 96.


In [83]:
from langchain_mcp_adapters.client import MultiServerMCPClient

async with MultiServerMCPClient(
        {
            "math":{
                "url": "http://localhost:8001/sse",
                "transport": "sse",
            },
            "weather": {
                "url": "http://localhost:8002/sse",
                "transport": "sse",
            },
        }) as client:
    tools = client.get_tools()    
    agent = create_react_agent(model=model, tools=client.get_tools())
    math_wheather_response = await agent.ainvoke({"messages": "what's (3 + 5) x 12? and what is the weather in nyc?"})


for m in math_wheather_response["messages"]:
    m.pretty_print(), 

================================ Human Message =================================

what's (3 + 5) x 12? and what is the weather in nyc?
================================== Ai Message ==================================
Tool Calls:
  add (call_lm4WJoiMirkd5eUDmSypSND9)
 Call ID: call_lm4WJoiMirkd5eUDmSypSND9
  Args:
    a: 3
    b: 5
  get_current_weather (call_VF1nmlwicvKLnujC3h250BYP)
 Call ID: call_VF1nmlwicvKLnujC3h250BYP
  Args:
    location: New York City
================================= Tool Message =================================
Name: add

8
================================= Tool Message =================================
Name: get_current_weather

{
  "location": "New York City",
  "temperature_c": 25,
  "condition": "Sunny",
  "humidity_pct": 40
}
================================== Ai Message ==================================
Tool Calls:
  multiply (call_ejjdodZbw3CmCf6yuwDq8AfY)
 Call ID: call_ejjdodZbw3CmCf6yuwDq8AfY
  Args:
    a: 8
    b: 12
==============================